In [1]:
import os, cv2, matplotlib as plt, numpy as np, pandas as pd,csv,glob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import array
import tensorflow as tf
import copy
from tensorflow import keras
from tensorflow.keras import layers
import PIL
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython.display import Image
from tensorflow.keras.callbacks import TensorBoard
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
data_path = 'D:\Masters project\CK+48'
data_dir_list = os.listdir(data_path)

img=tf.keras.preprocessing.image_dataset_from_directory(data_path, labels='inferred',label_mode='categorical',image_size=(48,
    48))

Found 750 files belonging to 5 classes.


In [3]:
img=img.shuffle(32)

In [4]:

test_ds = img.enumerate() \
                    .filter(lambda x,y: x % 4 == 0) \
                    .map(lambda x,y: y)

train_ds = img.enumerate() \
                    .filter(lambda x,y: x % 4 != 0) \
                    .map(lambda x,y: y)


In [5]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  #layers.experimental.preprocessing.RandomRotation(0.4),
])

In [6]:
base_model = tf.keras.applications.ResNet50(input_shape=(48,48,3),
                                               include_top=False,
                                               weights='imagenet')

In [7]:
base_model.trainable = True

In [8]:
inputs = keras.Input(shape=(48, 48,3))
x = data_augmentation(inputs)

In [9]:

x = base_model(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Flatten()(x)
x= keras.layers.Dense(128,activation='relu')(x)
#x = keras.layers.GlobalAveragePooling2D(2,2)(x)
x = keras.layers.Dropout(0.2)(x)
x= keras.layers.Dense(64,activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(5,activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
sequential (Sequential)      (None, 48, 48, 3)         0         
_________________________________________________________________
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 2048)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272

In [10]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(), # default from_logits=False
              metrics=[keras.metrics.CategoricalAccuracy()])

In [11]:

model.fit(train_ds, epochs=50)

Epoch 1/50
18/18 [==============================] - 9s 114ms/step - loss: 2.4012 - categorical_accuracy: 0.4119
Epoch 2/50
18/18 [==============================] - 1s 61ms/step - loss: 1.2903 - categorical_accuracy: 0.6683
Epoch 3/50
18/18 [==============================] - 1s 61ms/step - loss: 0.6935 - categorical_accuracy: 0.7360
Epoch 4/50
18/18 [==============================] - 1s 61ms/step - loss: 0.4823 - categorical_accuracy: 0.8387 0s - loss: 0.4874 - cate
Epoch 5/50
18/18 [==============================] - 2s 63ms/step - loss: 0.4297 - categorical_accuracy: 0.8388
Epoch 6/50
18/18 [==============================] - 1s 62ms/step - loss: 0.3972 - categorical_accuracy: 0.8527
Epoch 7/50
18/18 [==============================] - 1s 61ms/step - loss: 0.3438 - categorical_accuracy: 0.8854
Epoch 8/50
18/18 [==============================] - 2s 66ms/step - loss: 0.3508 - categorical_accuracy: 0.8716
Epoch 9/50
18/18 [==============================] - 2s 66ms/step - loss: 0.1955 - cate

In [12]:
y_train=tf.concat([y for x, y in train_ds], axis=0)
y_test=tf.concat([y for x, y in test_ds], axis=0)

In [13]:
#y_train

In [14]:

train_pred = model.predict(train_ds,verbose=1)
y_train_ground = tf.argmax(y_train, axis=1)
y_train_pred = tf.argmax(train_pred, axis=1)

18/18 [==============================] - 2s 26ms/step


In [15]:
from sklearn.metrics import accuracy_score,confusion_matrix
train_cmat = confusion_matrix(y_train_pred,y_train_ground)
print(train_cmat)

[[19  8 20 10 31]
 [11  7 20  7 20]
 [29 14 42 15 51]
 [16 10 17  7 30]
 [18 18 61 22 55]]


In [18]:
model.evaluate(test_ds)

6/6 [==============================] - 1s 27ms/step - loss: 0.2093 - categorical_accuracy: 0.9583


[0.20927156507968903, 0.9583333134651184]